In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.fpm import FPGrowth

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
products = spark.read.csv('files/products.csv', header=True, inferSchema=True)
orders = spark.read.csv('files/order_products__train.csv', header=True, inferSchema=True)

products.printSchema()
orders.printSchema()

In [ ]:
# Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in
# each transaction (and not only their ids). Then, group by the orders by 'order_id' to obtain one row per basket (
# i.e., set of products purchased together by one customer).


join = orders.join(products, orders.product_id == products.product_id)
df = join.groupBy("order_id").agg(collect_list("product_name").alias("products"))
df.show()

In [ ]:
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(df)

# Display frequent itemsets.
# Compute how many frequent itemsets and association rules were generated by running
# FP-growth alongside visalizing top frequent itemsets and association rules.


model.freqItemsets.show()
print("Number of frequent itemsets: " + str(model.freqItemsets.count()))
model.associationRules.show()
print("Number of association rules: " + str(model.associationRules.count()))